---

In [ ]:
"""
Python 3.8.17, using Conda env: PyRadiomics
@Author : Hasan Shaikh
@Email  : hasanshaikh3198@gmail.com
@GitHub : https://github.com/hash123shaikh
"""

---

## **CELL 1: Installation**

##### Description: Install required packages for radiomics analysis
##### Run this cell ONCE at the beginning, then restart your kernel

In [ ]:
!pip install pyradiomics
!pip install rt-utils
!pip install SimpleITK
!pip install pydicom
!pip install pandas
!pip install numpy

##### After running this cell, go to: Kernel -> Restart Kernel

---

## **CELL 2: Import Libraries**

##### Description: Import all necessary Python libraries
##### These libraries help us work with medical images and extract features

In [59]:
import os
import SimpleITK as sitk                      # For medical image processing
import pydicom                                # For reading DICOM files
from radiomics import featureextractor        # For extracting radiomics features
import pandas as pd                           # For organizing data in tables
import numpy as np                            # For numerical operations
from pathlib import Path                      # For handling file paths
import rt_utils                               # For handling RTSTRUCT contours

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


---

## **CELL 3: Set Working Directory**

##### Description: Set the folder where your patient data is located
##### TODO: Change this to YOUR data folder path

In [60]:
# TODO: CHANGE THIS PATH to where your data is stored
base_folder = "/home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images"  # Example: "/home/user/MAASTRO Dataset/Images/"

# Change working directory
os.chdir(base_folder)
print(f"Working directory set to: {os.getcwd()}")

Working directory set to: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images


---

### **CELL 4a: Configuration Parameters**

##### Description: Set the parameters for your analysis
##### TODO: Modify these settings according to your needs


### **CELL 4b: Understanding Feature Modes (INFO)**

##### Different feature extraction modes:

##### "original" - ~100 features (FAST, ~5-10 sec/patient)
   - Shape features (14)
   - First-order statistics (18)
   - Texture features: GLCM, GLRLM, GLSZM, GLDM, NGTDM (~70)

##### "wavelet" - ~800 features (SLOW, ~30-60 sec/patient)
   - Original features PLUS
   - 8 wavelet decompositions (HHH, HHL, HLH, etc.)
   - Each decomposition extracts first-order + texture features

##### "log" - ~200 features (MEDIUM, ~15-20 sec/patient)
   - Original features PLUS
   - Laplacian of Gaussian filtered images (5 sigma values)
   - Better for detecting edges and boundaries

##### "all" - 1000+ features (VERY SLOW, ~60-120 sec/patient)
   - All of the above plus square, square root, etc.

##### Recommendation: Start with "original" for your first run!


In [ ]:
# TODO: Set the ROI (Region of Interest) name you want to analyze
TARGET_ROI = "GTV-1"  # Common examples: "GTV-1", "GTV", "CTV", "PTV"

# TODO: Set the output CSV filename
OUTPUT_CSV = "radiomics_features.csv"

# TODO: Choose feature extraction mode
# Options: "original", "wavelet", "log", "all"
FEATURE_MODE = "all"                   # <-- CHANGE if needed


# Display settings
print("="*60)
print("CONFIGURATION SETTINGS")
print("="*60)
print(f"Target ROI: {TARGET_ROI}")
print(f"Output CSV: {OUTPUT_CSV}")
print(f"Feature mode: {FEATURE_MODE}")
print(f"Base folder: {base_folder}")
print("="*60)

CONFIGURATION SETTINGS
Target ROI: GTV-1
Output CSV: radiomics_features.csv
Base folder: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images


---

## **CELL 5: Function to Load CT Images**

##### Description: This function reads all CT DICOM slices from a folder and combines them into a single 3D image volume

In [48]:
def load_ct_series(ct_folder):
    """
    Load CT DICOM series from folder
    
    Args:
        ct_folder: Path to folder containing CT DICOM files
        
    Returns:
        ct_image: 3D CT image as SimpleITK image object
    """
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(ct_folder)
    reader.SetFileNames(dicom_names)
    ct_image = reader.Execute()
    return ct_image

print("✓ Function 'load_ct_series' defined")

✓ Function 'load_ct_series' defined


---

## **CELL 6: Function to Find ROI in RTSTRUCT Files**

##### Description: This function searches through RTSTRUCT files to find the one containing your target ROI (like GTV-1)
 * *Note: A patient may have multiple RTSTRUCT files with different contours*

In [49]:
def find_roi_in_rtstructs(rtstruct_folder, ct_folder, target_roi_name="GTV-1"):
    """
    Search through all RTSTRUCT files to find the one containing target ROI
    
    Args:
        rtstruct_folder: Path to folder with RTSTRUCT files
        ct_folder: Path to CT folder (needed for spatial registration)
        target_roi_name: Name of ROI to search for (e.g., "GTV-1")
        
    Returns:
        rt_struct: The RTSTRUCT object containing the target ROI
        target_roi_name: Name of the found ROI
    """
    rtstruct_files = list(Path(rtstruct_folder).glob("*.dcm"))
    
    if not rtstruct_files:
        raise FileNotFoundError(f"No RTSTRUCT files found in {rtstruct_folder}")
    
    print(f"Found {len(rtstruct_files)} RTSTRUCT file(s)")
    
    for rtstruct_file in rtstruct_files:
        try:
            print(f"  Checking: {rtstruct_file.name}")
            rt_struct = rt_utils.RTStructBuilder.create_from(
                dicom_series_path=str(ct_folder),
                rt_struct_path=str(rtstruct_file)
            )
            
            roi_names = rt_struct.get_roi_names()
            print(f"    Available ROIs: {roi_names}")
            
            # Check if target ROI exists in this RTSTRUCT
            if target_roi_name in roi_names:
                print(f"    ✓ Found '{target_roi_name}' in {rtstruct_file.name}")
                return rt_struct, target_roi_name
                
        except Exception as e:
            print(f"    Error reading {rtstruct_file.name}: {str(e)}")
            continue
    
    raise ValueError(f"ROI '{target_roi_name}' not found in any RTSTRUCT file")

print("✓ Function 'find_roi_in_rtstructs' defined")

✓ Function 'find_roi_in_rtstructs' defined


---

## **CELL 7: Function to Convert RTSTRUCT to Binary Mask**

##### Description: This function converts the contour from RTSTRUCT into a binary mask (0s and 1s) that matches the CT image dimensions

In [50]:
def load_rtstruct_as_mask(rt_struct, ct_image, roi_name):
    """
    Convert RTSTRUCT ROI to binary mask
    
    Args:
        rt_struct: RTSTRUCT object containing the contour
        ct_image: CT image to match dimensions with
        roi_name: Name of the ROI to extract
        
    Returns:
        mask_image: Binary mask (0 = background, 1 = tumor/ROI)
    """
    print(f"Extracting mask for ROI: {roi_name}")
    
    # Get mask as numpy array from rt_utils
    mask_array = rt_struct.get_roi_mask_by_name(roi_name)
    
    # rt_utils returns mask in (Z, Y, X) format
    # but SimpleITK expects the array to match CT image dimensions
    # Need to transpose to match CT image axis order
    print(f"  Original mask shape: {mask_array.shape}")
    print(f"  CT image size: {ct_image.GetSize()}")
    
    # Transpose if dimensions don't match (axis ordering)
    # rt_utils: (Z, Y, X) -> SimpleITK needs: same as CT which is (X, Y, Z)
    # GetImageFromArray reads as (Z, Y, X), so we need mask in that order
    if mask_array.shape != ct_image.GetSize()[::-1]:
        # Transpose from (Z, Y, X) to match CT image axis order
        mask_array = np.transpose(mask_array, (2, 1, 0))
        print(f"  Transposed mask shape: {mask_array.shape}")
    
    # Convert to SimpleITK image
    mask_image = sitk.GetImageFromArray(mask_array.astype(np.uint8))
    
    # Copy spatial information from CT (spacing, origin, direction)
    mask_image.SetSpacing(ct_image.GetSpacing())
    mask_image.SetOrigin(ct_image.GetOrigin())
    mask_image.SetDirection(ct_image.GetDirection())
    
    # Verify dimensions match
    if mask_image.GetSize() != ct_image.GetSize():
        raise ValueError(
            f"Mask size {mask_image.GetSize()} doesn't match CT size {ct_image.GetSize()}"
        )
    
    print(f"  Final mask size: {mask_image.GetSize()}")
    
    return mask_image

print("✓ Function 'load_rtstruct_as_mask' defined")

✓ Function 'load_rtstruct_as_mask' defined


---

## **CELL 8: Function to Extract Radiomics Features**

##### Description: This is the core function that extracts radiomics features
##### It calculates shape, intensity, and texture features from the ROI


In [ ]:
def extract_radiomics_features(ct_image, mask_image, patient_id, roi_name, feature_mode="original"):
    """
    Extract radiomics features using PyRadiomics
    
    Features extracted include:
    - Shape features: volume, surface area, sphericity, etc.
    - First-order features: mean, median, skewness, kurtosis, etc.
    - Texture features: GLCM, GLRLM, GLSZM, GLDM, NGTDM
    
    Args:
        ct_image: CT image volume
        mask_image: Binary mask of ROI
        patient_id: Patient identifier
        roi_name: Name of the ROI
        
    Returns:
        feature_dict: Dictionary containing all extracted features
    """
    # Initialize feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()

    # Configure based on feature mode
    if feature_mode == "wavelet":
        print("  Enabling wavelet transforms...")
        extractor.enableImageTypeByName('Wavelet')
        
    elif feature_mode == "log":
        print("  Enabling LoG filters...")
        extractor.enableImageTypeByName('LoG')
        
    elif feature_mode == "all":
        print("  Enabling ALL image types (wavelet, LoG, square, etc.)...")
        extractor.enableAllImageTypes()
    
    else:  # "original"
        print("  Using original image only...")
    

    print("Extracting radiomics features...")
    features = extractor.execute(ct_image, mask_image) # Extract features
    
    # Convert to dictionary with patient info
    feature_dict = {
        'PatientID': patient_id,
        'ROI_Name': roi_name
    }
    
    # Add all extracted features (skip diagnostic info)
    for key, value in features.items():
        if not key.startswith('diagnostics_'):
            feature_dict[key] = value
    
    return feature_dict

print("✓ Function 'extract_radiomics_features' defined")

✓ Function 'extract_radiomics_features' defined


---

## **CELL 9: Function to Process Single Patient**

##### Description: This function processes ONE patient's data
##### It combines all the previous functions to:
 * 1. Load CT
 * 2. Find RTSTRUCT with target ROI
 * 3. Create mask
 * 4. Extract features

In [ ]:
def process_patient_folder(patient_folder, target_roi="GTV-1", feature_mode="original"):
    """
    Process single patient folder and extract features for specific ROI
    
    Expected folder structure:
    patient_folder/
        ├── CT/          (or CT SCAN)
        └── RTSTRUCT/
    
    Args:
        patient_folder: Path to patient folder
        target_roi: ROI name to extract (default: "GTV-1")
        feature_mode: Feature extraction mode (default: "original")
        
    Returns:
        features: Dictionary of extracted features
    """

    patient_id = Path(patient_folder).name
    print(f"\n{'='*60}")
    print(f"Processing patient: {patient_id}")
    print(f"{'='*60}")
    
    # TODO: Modify these folder names if your structure is different
    ct_folder = Path(patient_folder) / "CT"  # Change to "CT SCAN" if needed
    rtstruct_folder = Path(patient_folder) / "RTSTRUCT"
    
    # Validate folders exist
    if not ct_folder.exists():
        print(f"❌ CT folder not found for {patient_id}")
        return None
    
    if not rtstruct_folder.exists():
        print(f"❌ RTSTRUCT folder not found for {patient_id}")
        return None
    
    try:
        # Step 1: Load CT series
        print("Loading CT series...")
        ct_image = load_ct_series(str(ct_folder))
        print(f"  CT Image size: {ct_image.GetSize()}")
        
        # Step 2: Find RTSTRUCT containing target ROI
        print(f"Searching for ROI '{target_roi}'...")
        rt_struct, roi_name = find_roi_in_rtstructs(
            rtstruct_folder, 
            ct_folder, 
            target_roi
        )
        
        # Step 3: Convert RTSTRUCT to mask
        mask_image = load_rtstruct_as_mask(rt_struct, ct_image, roi_name)
        print(f"  Mask created successfully")
        
        # Step 4: Extract features
        features = extract_radiomics_features(ct_image, mask_image, patient_id, roi_name, feature_mode)
        print(f"✓ Successfully extracted {len(features)-2} features")
        
        return features
        
    except Exception as e:
        print(f"❌ Error processing {patient_id}: {str(e)}")
        return None

print("✓ Function 'process_patient_folder' defined")

✓ Function 'process_patient_folder' defined


---

## **CELL 10: Function to Process Multiple Patients (Batch Processing)**

##### Description: This function processes ALL patients in a folder
##### It loops through each patient and saves results to CSV


In [ ]:
def process_multiple_patients(base_folder, output_csv="radiomics_features.csv", target_roi="GTV-1", feature_mode="original"):
    """
    Process multiple patient folders and save to CSV
    
    Args:
        base_folder: Path to folder containing all patient folders
        output_csv: Output CSV filename
        target_roi: ROI name to extract (default: "GTV-1")
        
    Returns:
        df: Pandas DataFrame with all features
    """

    all_features = []
    failed_patients = []
    
    # Get all patient folders
    patient_folders = sorted([f for f in Path(base_folder).iterdir() if f.is_dir()])
    
    print(f"\n{'='*60}")
    print(f"BATCH PROCESSING: {len(patient_folders)} patients")
    print(f"Target ROI: {target_roi}")
    print(f"Mode: {feature_mode}")
    print(f"{'='*60}\n")
    
    # Loop through each patient
    for idx, patient_folder in enumerate(patient_folders, 1):
        print(f"\n[{idx}/{len(patient_folders)}] ", end="")
        
        try:
            features = process_patient_folder(str(patient_folder), target_roi, feature_mode)
            if features:
                all_features.append(features)
            else:
                failed_patients.append(patient_folder.name)
        except Exception as e:
            print(f"❌ Unexpected error: {str(e)}")
            failed_patients.append(patient_folder.name)
            continue
    
    # Convert to DataFrame and save
    print(f"\n{'='*60}")
    print("SUMMARY")
    print(f"{'='*60}")
    
    if all_features:
        df = pd.DataFrame(all_features)
        df.to_csv(output_csv, index=False)
        
        print(f"✓ Successfully processed: {len(all_features)} patients")
        print(f"✓ Features saved to: {output_csv}")
        print(f"✓ Total features per patient: {len(df.columns) - 2}")
        
        if failed_patients:
            print(f"\n⚠ Failed patients ({len(failed_patients)}):")
            for patient in failed_patients:
                print(f"  - {patient}")
        
        return df
    else:
        print("❌ No features extracted from any patient")
        if failed_patients:
            print(f"\nFailed patients ({len(failed_patients)}):")
            for patient in failed_patients:
                print(f"  - {patient}")
        return None

print("✓ Function 'process_multiple_patients' defined")


✓ Function 'process_multiple_patients' defined


----

## **CELL 11: TEST - Process Single Patient (Optional)**

##### Description: Use this cell to test with ONE patient first
##### This helps verify everything works before processing all patients
##### Uncomment the code below to test

In [ ]:
# TODO: UNCOMMENT and change patient folder name to test
# test_patient = "HN1004"  # Change this to your patient ID
# features = process_patient_folder(test_patient, target_roi=TARGET_ROI, feature_mode=FEATURE_MODE)
# 
# if features:
#     # Save single patient results
#     df_test = pd.DataFrame([features])
#     df_test.to_csv(f"{test_patient}_test.csv", index=False)
#     print(f"\n✓ Test successful! Features saved to {test_patient}_test.csv")
#     print(f"\nNumber of features extracted: {len(features)-2}")
#     
#     # Display first few features
#     print("\nSample features:")
#     for i, (key, value) in enumerate(list(features.items())[:10]):
#         print(f"  {key}: {value}")

---

## **CELL 12: MAIN - Process All Patients**

##### Description: This is the main cell that processes ALL patients
##### Make sure you've tested with one patient first (Cell 11)
##### TODO: Verify your settings before running!

In [ ]:
# TODO: Verify these settings before running
print("Current Settings:")
print(f"  Base folder: {base_folder}")
print(f"  Target ROI: {TARGET_ROI}")
print(f"  Mode: {FEATURE_MODE}")
print(f"  Output file: {OUTPUT_CSV}")

Current Settings:
  Base folder: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images
  Target ROI: GTV-1
  Output file: radiomics_features.csv

Ready to process? If settings look correct, run the next cell!


---

## **CELL 13: Execute Batch Processing**

##### Description: Run this cell to start processing all patients
##### This may take several minutes depending on number of patients


In [ ]:
# Process all patients
df = process_multiple_patients(
    base_folder=base_folder,
    output_csv=OUTPUT_CSV,
    target_roi=TARGET_ROI,
    feature_mode=FEATURE_MODE
)

if df is not None:
    print(f"\n{'='*60}")
    print("✓ PROCESSING COMPLETE!")
    print(f"{'='*60}")
    print(f"Results saved to: {OUTPUT_CSV}")
    print(f"Total patients processed: {len(df)}")
    print(f"Total features per patient: {len(df.columns)}")

---

## **CELL 14: View Results**

##### Description: Display and explore the extracted features

In [57]:
if df is not None:
    print("Dataset Shape:", df.shape)
    print("\n" + "="*60)
    print("FIRST FEW PATIENTS:")
    print("="*60)
    display(df.head())
    
    print("\n" + "="*60)
    print("FEATURE NAMES (first 20):")
    print("="*60)
    for i, col in enumerate(df.columns[:20], 1):
        print(f"{i}. {col}")
    
    print(f"\n... and {len(df.columns)-20} more features")

Dataset Shape: (137, 109)

FIRST FEW PATIENTS:


,PatientID,ROI_Name,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,HN1004,GTV-1,0.759295,0.579260,43.402040,74.926725,87.2084652212731,87.2084652212731,72.90285978843376,92.06265546755797,...,1287.5957851586468,0.006091649950355723,7.704573394254903,0.15083447778217454,25382.1451931632,0.7386569363993765,0.00022174936904843307,8160.791193641188,0.0920944233175849,2.92934665418536
1,HN1006,GTV-1,0.696800,0.301773,11.687646,38.729977,29.058468512383033,38.42760289248472,34.897594502354764,41.98898970382184,...,1503.5185456526194,0.002805847957274179,6.649553739899192,0.22580645161290322,392.485794317727,0.04427043430246803,0.005246835265013618,3826.6394079165557,0.08127741751771043,21.14690716471923
2,HN1022,GTV-1,0.783281,0.560082,30.254456,54.017881,58.866938142305315,66.64742235263117,47.37083138919884,67.27194638560573,...,1366.0902545482081,0.0034646931386199662,7.280442050718658,0.14339006761482864,9401.43455350651,0.25935123060427023,0.0006516007393318773,4733.3261968478455,0.02992862289795048,4.86061361915349
3,HN1026,GTV-1,0.840406,0.744783,16.279669,21.858284,25.851043078039545,25.604232485888076,22.860742013124266,27.426260555650057,...,26.76499981426582,0.034304406157078166,4.925242942144948,0.0632996632996633,2879.5229741964686,1.5148683536866485,0.006854360363610231,36.20038449686023,0.07403125368824401,0.37204523859057326
4,HN1029,GTV-1,0.610856,0.319918,13.239173,41.382967,40.95692525767163,40.95692525767163,28.721564784868637,44.40895281733213,...,721.9929663802085,0.006384503538529918,6.967953904562411,0.2803406671398155,417.1315750680981,0.6303783413384101,0.0021675426310988686,7336.805704692472,0.4004345551616009,9.178849012368433



FEATURE NAMES (first 20):
1. PatientID
2. ROI_Name
3. original_shape_Elongation
4. original_shape_Flatness
5. original_shape_LeastAxisLength
6. original_shape_MajorAxisLength
7. original_shape_Maximum2DDiameterColumn
8. original_shape_Maximum2DDiameterRow
9. original_shape_Maximum2DDiameterSlice
10. original_shape_Maximum3DDiameter
11. original_shape_MeshVolume
12. original_shape_MinorAxisLength
13. original_shape_Sphericity
14. original_shape_SurfaceArea
15. original_shape_SurfaceVolumeRatio
16. original_shape_VoxelVolume
17. original_firstorder_10Percentile
18. original_firstorder_90Percentile
19. original_firstorder_Energy
20. original_firstorder_Entropy

... and 89 more features


---

#### **END OF NOTEBOOK**

##### Congratulations! You've completed the radiomics feature extraction.

##### Next steps:
  * 1. Check your output CSV file
  * 2. Explore feature correlations
  * 3. Use features for machine learning models
  * 4. Perform survival analysis or outcome prediction

##### For questions or issues, refer to:
##### - PyRadiomics documentation: https://pyradiomics.readthedocs.io/

---